# Part 1 - Getting used to the new refractored methods

In [1]:
import sys
import os
import torch
from torch import nn
import numpy as np
import torch.optim as optim


# Get the absolute path of the directory containing the module you want to import
module_directory = r"C:\Users\Neel Patel\Documents\Github Repositories\WOD Project\WOD-prediction"

# Add the directory to the system path
sys.path.append(module_directory)

import wod_predictor
from wod_predictor.data_loader import DataLoader
from wod_predictor.splitter import DataSpitter
from wod_predictor.preprocessor import DataPreprocessor 
from wod_predictor.modeling import RandomForestModel,NeuralNetV0

In [2]:
data_path = wod_predictor.__path__[0].replace("wod_predictor", "Data")
loader = DataLoader(root_path = data_path, objects= ['open_results','descriptions','benchmark_stats'])
data = loader.load()

c:\users\neel patel\documents\github repositories\wod project\wod-prediction\wod_predictor\data_loader.py:22: DtypeWarning: Columns (2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))
c:\users\neel patel\documents\github repositories\wod project\wod-prediction\wod_predictor\data_loader.py:22: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(self.root_path, file))


In [3]:
data['open_results']

,19.1,19.2,19.3,19.4,19.5,20.1,20.2,20.3,20.4,20.5,...,21.2,21.3,21.4,22.1,22.2,22.3,23.1,23.2A,23.2B,23.3
id,,,,,,,,,,,,,,,,,,,,,
153604,387 reps,16:28,7:27,8:08,6:53,8:28,959 reps,6:08,14:22,10:45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81616,399 reps,18:06,8:29,8:41,7:57,8:16,991 reps,6:40,17:47,10:11,...,9:39,8:19,275 lbs,352 reps,8:52,4:41,291 reps,177 reps,276 lbs,9:15
199938,380 reps,18:04,7:38,8:23,7:21,9:18,962 reps,7:22,17:56,10:48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
514502,388 reps,17:06,9:29,8:33,7:20,8:05,1021 reps,6:32,15:00,11:11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308712,399 reps,18:26,7:40,9:14,8:12,9:02,1000 reps,6:52,16:52,11:07,...,NaN,NaN,NaN,353 reps,9:08,5:17,268 reps,168 reps,272 lbs,9:41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2331240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1710035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data['benchmark_stats']

,name,Back Squat,Chad1000x,Clean and Jerk,Deadlift,Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Max Pull-ups,Run 5k,Snatch,Sprint 400m
id,,,,,,,,,,,,,,,
469656,Jeffrey Adler,475.0,NaN,377.0,567.0,NaN,917.0,122.0,76.0,438.0,NaN,54.0,1155.0,290.0,59.0
300638,Tola Morakinyo,500.0,NaN,390.0,615.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,340.0,NaN
676693,Colten Mertens,555.0,NaN,335.0,545.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,275.0,NaN
663689,Tyler Christophel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1031875,Roldan Goldbaum,445.0,NaN,350.0,515.0,NaN,NaN,130.0,67.0,NaN,NaN,NaN,1080.0,280.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594361,Erik Davidson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1768926,Ammar El-wardani,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
205540,Doug Shinstrom,400.0,NaN,297.0,455.0,NaN,NaN,193.0,133.0,NaN,NaN,25.0,NaN,215.0,98.0


In [5]:
splitter = DataSpitter(sample = 1000, test_ratio=  0.2, test_filter='23.*')
train_data, test_data = splitter.split(data)

In [6]:
train_data['benchmark_stats'].shape,test_data['benchmark_stats'].shape

((800, 15), (200, 15))

In [7]:
train_data['open_results'].shape,test_data['open_results'].shape

((800, 21), (200, 4))

In [8]:
train_data['open_results']

,19.1,19.2,19.3,19.4,19.5,20.1,20.2,20.3,20.4,20.5,...,21.2,21.3,21.4,22.1,22.2,22.3,23.1,23.2A,23.2B,23.3
id,,,,,,,,,,,,,,,,,,,,,
2158,241 reps,169 reps,112 reps,116 reps,19:03,172 reps,581 reps,NaN,NaN,NaN,...,16:20,150 reps,221 lbs,202 reps,144 reps,162 reps,180 reps,128 reps,206 lbs,129 reps
2502,NaN,340 reps,141 reps,11:40,NaN,13:59,680 reps,115 reps,200 reps,17:51,...,12:04,11:17,265 lbs,270 reps,111 reps,8:57,194 reps,155 reps,247 lbs,210 reps
4307,349 reps,259 reps,118 reps,NaN,NaN,10:56,715 reps,NaN,157 reps,14:55,...,11:40,14:41,230 lbs,300 reps,9:46,7:53,195 reps,151 reps,210 lbs,211 reps
5399,329 reps,343 reps,151 reps,11:19,11:20,11:54,853 reps,8:47,204 reps,13:24,...,12:52,12:46,245 lbs,305 reps,9:53,7:28,224 reps,150 reps,260 lbs,11:46
8374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,212 reps,157 reps,237 lbs,286 reps
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,246 reps,145 reps,253 lbs,212 reps
2512976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,180 reps,130 reps,188 lbs,128 reps
2513668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,207 reps,168 reps,253 lbs,286 reps


In [9]:
keys_desc = train_data['workout_descriptions'].keys()

for key in keys_desc:
    if train_data['workout_descriptions'][key]['goal'].lower() == "load":
        print(train_data['workout_descriptions'][key])
    

{'goal': 'Load', 'time_cap': 12, 'total_reps': 2, 'description': 'Dumbbell squat, Bar-facing burpees, 1-rep-max clean, Men use 50-lb. dumbbells'}
{'goal': 'Load', 'time_cap': 7, 'total_reps': 4, 'description': '1 deadlift, 1 clean, 1 hang clean, 1 jerk, Time begins immediately following the completion of 21.3.'}
{'goal': 'Load', 'time_cap': 5, 'total_reps': 1, 'description': '1-rep-max thruster (from the floor)'}


In [10]:
test_data['workout_descriptions'].keys()

dict_keys(['17.1', '17.2', '17.3', '17.4', '17.5', '18.1', '18.2', '18.2a', '18.3', '18.4', '18.5', '19.1', '19.2', '19.3', '19.4', '19.5', '20.1', '20.2', '20.3', '20.4', '20.5', '21.1', '21.2', '21.3', '21.4', '22.1', '22.2', '22.3', '23.1', '23.2A', '23.2B', '23.3', '24.1'])

In [11]:
# # Check for "--" in the open_results dataset
# for col in train_data['open_results'].columns:
#     if train_data['open_results'][col].dtype == "object":  # Only check object/string columns
#         if train_data['open_results'][col].str.contains('--').any():
#             print(f"Column '{col}' contains '--' values.")
            
# # Check for "--" in the open_results dataset and show specific elements
# for col in train_data['open_results'].columns:
#     if train_data['open_results'][col].dtype == "object":  # Only check object/string columns
#         rows_with_dashes = train_data['open_results'][col].str.contains('--', na=False)
#         if rows_with_dashes.any():
#             print(f"Column '{col}' contains '--' values at the following rows:")
#             print(train_data['open_results'][rows_with_dashes][col])  # Show the rows with '--'

# train_data['open_results']

# Part 2 - Creating the Neural Network using the refactored methods

In [12]:
preprocessing_config = {
    'open_results':
    {
        'scale_up': True,
        'scale_method': 'standard', # standard or quantile

    },
    'benchmark_stats':{
        'remove_outliers': True,
        'missing_method':'zero'
    },
    # 'athlete_info':{}
}
preprocessor = DataPreprocessor(config=preprocessing_config)
preprocessor.fit(data=train_data)

In [13]:
train_preprocessed = preprocessor.transform(data = train_data)
test_preprocessed = preprocessor.transform(data=test_data)
train_preprocessed

c:\users\neel patel\documents\github repositories\wod project\wod-prediction\wod_predictor\feature_engineering_parts\open_results_fe.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  workout_ids = workout_ids.str.replace(".", "_")
c:\users\neel patel\documents\github repositories\wod project\wod-prediction\wod_predictor\feature_engineering_parts\open_results_fe.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  workout_ids = workout_ids.str.replace(".", "_")


{'X':                workout_19.1  workout_19.2  workout_19.3  workout_19.4  \
 2158_19_1                 1             0             0             0   
 4307_19_1                 1             0             0             0   
 5399_19_1                 1             0             0             0   
 8700_19_1                 1             0             0             0   
 12283_19_1                1             0             0             0   
 ...                     ...           ...           ...           ...   
 896315_23_1s              0             0             0             0   
 1301224_23_1s             0             0             0             0   
 2248185_23_1s             0             0             0             0   
 2490528_23_3s             0             0             0             0   
 2495065_23_3s             0             0             0             0   
 
                workout_19.5  workout_20.1  workout_20.2  workout_20.3  \
 2158_19_1                 0   

In [14]:
train_preprocessed['X'].head(20)

,workout_19.1,workout_19.2,workout_19.3,workout_19.4,workout_19.5,workout_20.1,workout_20.2,workout_20.3,workout_20.4,workout_20.5,...,Fight Gone Bad,Filthy 50,Fran,Grace,Helen,L1 Benchmark,Max Pull-ups,Run 5k,Snatch,Sprint 400m
2158_19_1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,124.0,106.0,453.0,0.0,50.0,0.0,225.00000,0.0
4307_19_1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
5399_19_1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,126.0,90.0,451.0,0.0,75.0,1200.0,265.00000,64.0
8700_19_1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,174.0,102.0,0.0,0.0,40.0,1372.0,230.00000,0.0
12283_19_1,1,0,0,0,0,0,0,0,0,0,...,0.0,1238.0,149.0,111.0,519.0,0.0,0.0,1195.0,225.00000,0.0
15349_19_1,1,0,0,0,0,0,0,0,0,0,...,0.0,1171.0,162.0,110.0,476.0,0.0,50.0,1232.0,250.00000,72.0
18494_19_1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,233.0,250.0,624.0,0.0,40.0,1170.0,187.39270,0.0
21559_19_1,1,0,0,0,0,0,0,0,0,0,...,0.0,1531.0,145.0,210.0,497.0,0.0,50.0,0.0,211.64352,0.0
22128_19_1,1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,125.0,0.0,0.0,0.0,0.0,0.0,235.00000,0.0
22885_19_1,1,0,0,0,0,0,0,0,0,0,...,0.0,1180.0,137.0,124.0,492.0,0.0,79.0,1217.0,225.00000,58.0


In [15]:
X_train_torch = torch.tensor(train_preprocessed['X'].values,dtype = torch.float32)
y_train_torch = torch.tensor(train_preprocessed['y'].values,dtype = torch.float32).view(-1,1)

X_test_torch = torch.tensor(test_preprocessed['X'].values,dtype = torch.float32)
y_test_torch = torch.tensor(test_preprocessed['y'].values,dtype = torch.float32).view(-1,1)

In [16]:
# class WOD_predictor(nn.Module):
    
#     def __init__(self, input_features, hidden_units, output_features):
#         super(WOD_predictor, self).__init__()
#           #Create the layers
#         self.model = nn.Sequential(
#         nn.Linear(in_features = input_features, out_features = hidden_units),
#         nn.ReLU(),
#         nn.Linear(in_features = hidden_units, out_features = hidden_units),
#         nn.ReLU(),
#         nn.Linear(in_features = hidden_units, out_features = hidden_units),
#         nn.ReLU(),
#         #nn.Dropout(p=0.2),
#         nn.Linear(in_features = hidden_units, out_features = hidden_units),
#         nn.ReLU(),
#         nn.Linear(in_features = hidden_units, out_features = hidden_units),
#         nn.ReLU(),
#         nn.Linear(in_features = hidden_units, out_features = hidden_units),
#         nn.ReLU(),
#         nn.Linear(in_features = hidden_units, out_features = output_features),
#         )
        

#     def forward(self, x):
#         return self.model(x)

# model_V0 = WOD_predictor(input_features = X_train_torch.shape[1], output_features = 1, hidden_units = 64)
# loss_fn = nn.MSELoss()
# optimizer = torch.optim.Adam(params = model_V0.parameters(), lr = 0.001)


In [17]:
# torch.manual_seed(42)
# epochs = 1000

# for epoch in range(epochs):
#     model_V0.train()
    
    
#     y_pred = model_V0(X_train_torch)
#     loss = loss_fn(y_pred,y_train_torch)
    
#     #print(f'Ypred shape:{y_pred.shape}||y_train shape{y_train.shape}')
    
#     optimizer.zero_grad()
#     loss.backward()
    
#     optimizer.step()
    
#     ### Testing loop
    
#     model_V0.eval()
    
#     with torch.inference_mode():
#         test_pred = model_V0(X_test_torch)
#         test_loss = loss_fn(test_pred,y_test_torch)
#     if epoch%100 == 0:
#             print(f"Epoch: {epoch}| Training Loss: {loss} | Test Loss: {test_loss}")

In [18]:
# import pandas as pd
# from wod_predictor.models.helpers import unstack_series,show_breakdown_by_workout

# y_test = test_preprocessed['y']
# meta_data = test_preprocessed['meta_data']
# y_pred = test_pred


# y_test_unstacked = unstack_series(y_test, meta_data)
# y_pred_unstacked = unstack_series(
#     pd.Series(y_pred.detach().numpy().squeeze(), index=y_test.index, name="score"), meta_data
# )

# # reverse the scaling
# if "scaler" in meta_data:
#     y_test_unstacked = meta_data["scaler"].reverse(y_test_unstacked)
#     y_pred_unstacked = meta_data["scaler"].reverse(y_pred_unstacked)


# show_breakdown_by_workout(y_pred_unstacked, y_test_unstacked)

In [19]:
#y_test_unstacked

In [20]:
#y_pred_unstacked

In [21]:
config = {}
rf_modeler = RandomForestModel(config = config, n_estimators = 10)
rf_modeler.fit(X = train_preprocessed['X'], y = train_preprocessed['y'])
rf_modeler.predict(X = test_preprocessed['X'])
rf_modeler.show_results(y_test = test_preprocessed['y'], meta_data = test_preprocessed['meta_data'])

random forest method called
Mean Absolute Error: 0.81
Mean Absolute Percentage Error: 1.7


,y_test_mean,y_pred_mean,error_mean,error_percentage
workout_name,,,,
workout_23.1,184.680000,185.877932,8.055480,4.361859
workout_23.2A,132.370000,131.937499,12.731264,9.617938
workout_23.2B,208.125000,207.765735,19.420871,9.331349
workout_23.3,24.196140,26.417340,8.684917,35.893813
workout_23.3_scaled,7.166667,12.235035,5.068369,70.721424


In [22]:
#Using the class we made!

model_V0 = NeuralNetV0(input_features = X_train_torch.shape[1], output_features = 1, hidden_units = 64)
model_V0.fit(X_train_torch, y_train_torch, epochs=1000, lr=0.001)
y_pred = model_V0.predict(X_test_torch)
model_V0.show_results(y_test=test_preprocessed['y'], meta_data=test_preprocessed['meta_data'])


Epoch: 0 | Training Loss: 1.0504
Epoch: 100 | Training Loss: 0.9767
Epoch: 200 | Training Loss: 0.9574
Epoch: 300 | Training Loss: 0.9353
Epoch: 400 | Training Loss: 0.8832
Epoch: 500 | Training Loss: 0.8265
Epoch: 600 | Training Loss: 0.7869
Epoch: 700 | Training Loss: 0.7217
Epoch: 800 | Training Loss: 0.7113
Epoch: 900 | Training Loss: 0.6423
Predict method called
(792,)
Mean Absolute Error: 0.99
Mean Absolute Percentage Error: 5.33


,y_test_mean,y_pred_mean,error_mean,error_percentage
workout_name,,,,
workout_23.1,184.680000,185.657867,10.309977,5.582617
workout_23.2A,132.370000,132.164963,14.941074,11.287357
workout_23.2B,208.125000,207.340317,25.418159,12.212929
workout_23.3,24.196140,26.213497,40.296915,166.542742
workout_23.3_scaled,7.166667,12.230518,5.063852,70.658395
